## Import Libraries

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import models, layers, utils, losses, optimizers, initializers, regularizers
from keras.wrappers import scikit_learn
from keras.models import *
from keras.layers import *
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from keras.utils import np_utils, image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix

import random
import pandas as pd
import numpy as np
from PIL import Image, ImageFilter
import matplotlib.pyplot as plt

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

## Count the Number of Files, and Take Random Samples from the Image Files

In [2]:
# flist = list(pd.read_csv('flist.txt', header = None)[0])
flist = list(pd.read_csv('/kaggle/input/dissertation-1-data/Files/flist.txt', header = None)[0])

## Overview of the `properties` Dataset

Read the `properties` dataset first, and make sure that `property type` is a categorical variable.

In [3]:
# properties = pd.read_csv('properties.csv')
# properties_juny12 = pd.read_csv('properties_juny12.csv')
properties = pd.read_csv('/kaggle/input/dissertation-1-data/Files/properties.csv')
properties_juny12 = pd.read_csv('/kaggle/input/dissertation-1-data/Files/properties_juny12.csv')

properties_full = pd.concat([properties, properties_juny12])
properties = properties_full
properties.propertyType = properties.propertyType.astype('category')
properties.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37402 entries, 0 to 19851
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   Unnamed: 0    37402 non-null  int64   
 1   address       37402 non-null  object  
 2   propertyType  37402 non-null  category
 3   bedrooms      24486 non-null  float64 
 4   detailUrl     37402 non-null  object  
 5   location_lat  37402 non-null  float64 
 6   location_lng  37402 non-null  float64 
 7   property_id   37402 non-null  object  
dtypes: category(1), float64(3), int64(1), object(3)
memory usage: 2.3+ MB


## A Subset of the `properties` Dataset

As random samples of images have been obtained previously, a subset of the whole `properties` dataset could hence be formulated by selecting the rows of the whole `properties` dataset corresponding to the selected samples.

In [4]:
flist_id = list(map(lambda string: string[-40 : -4], flist))
properties_sub = pd.DataFrame(properties.loc[properties['property_id'].isin(flist_id)])
properties_sub = properties_sub.drop_duplicates(['location_lat', 'location_lng'])

# properties_sub = pd.read_csv('properties_sub.csv')
# properties_sub = pd.read_csv('/kaggle/input/dissertation-1-data/Files/properties_sub.csv')

properties_sub.propertyType = properties_sub.propertyType.astype('category')
flist_id = list(properties_sub.property_id)
properties_sub.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15484 entries, 0 to 19851
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   Unnamed: 0    15484 non-null  int64   
 1   address       15484 non-null  object  
 2   propertyType  15484 non-null  category
 3   bedrooms      10967 non-null  float64 
 4   detailUrl     15484 non-null  object  
 5   location_lat  15484 non-null  float64 
 6   location_lng  15484 non-null  float64 
 7   property_id   15484 non-null  object  
dtypes: category(1), float64(3), int64(1), object(3)
memory usage: 983.1+ KB


In [5]:
properties_sub.propertyType.value_counts(sort = False)

Detached         3337
Flat             2252
Semi-Detached    4062
Terraced         4124
Unknown          1709
Name: propertyType, dtype: int64

The original data should be splitted into training and testing sets, and the testing set contains 30% of the original data.

In [6]:
# directory = 'street_view/'
directory = '/kaggle/input/dissertation-1-data/Files/street_view/'

height = 64
width = 64
batch = 32

training = image_dataset_from_directory(
  directory,
  validation_split = 0.3,
  subset = 'training',
  seed = 123,
  image_size = (height, width),
  batch_size = batch,
  label_mode = 'categorical')

validation = image_dataset_from_directory(
  directory,
  validation_split = 0.3,
  subset = 'validation',
  seed = 123,
  image_size = (height, width),
  batch_size = batch,
  label_mode = 'categorical')

training = training.cache().prefetch(buffer_size = tf.data.AUTOTUNE)
validation = validation.cache().prefetch(buffer_size = tf.data.AUTOTUNE)

Found 13775 files belonging to 4 classes.
Using 9643 files for training.
Found 13775 files belonging to 4 classes.
Using 4132 files for validation.


In [7]:
propertyType_train_fac = np.argmax(np.asarray(list(training.unbatch().map(lambda x, y: y))), axis = 1)
propertyType_validation_fac = np.argmax(np.asarray(list(validation.unbatch().map(lambda x, y: y))), axis = 1)
labels = pd.Series(propertyType_train_fac).astype('category')

In [8]:
loss = losses.CategoricalCrossentropy()
weights = sum(labels.value_counts()) / (labels.value_counts(sort = False) * len(labels.cat.categories))
loss.weighted = weights

In [9]:
# sample_weights = weights[propertyType_train_fac]
# training_new = training.unbatch().batch(1).map(lambda x, y: (x, y, sample_weights))

In [10]:
Callbacks = [
             EarlyStopping(monitor = 'val_accuracy', mode = 'max', patience = 8, restore_best_weights = True), 
             ReduceLROnPlateau(monitor = 'val_accuracy', factor = 1e-1, patience = 0, cooldown = 0)
            ]

## Multi-Class Classification Using Neural Network

### Multi-Layer Perceptron (MLP) model

In [11]:
# mlp = Sequential([
#                   Rescaling(1. / 255, input_shape = (height, width, 3)),
#                   Flatten(),
#                   Dense(128, activation = tf.nn.leaky_relu),
#                   Dense(
#                         len(labels.cat.categories), 
#                         activation = tf.nn.softmax, 
#                         kernel_initializer = 'ones',
#                         kernel_regularizer = regularizers.L1(.01),
#                         activity_regularizer = regularizers.L1(.01)
#                        )
#                 ])
# mlp.compile(loss = loss, optimizer = keras.optimizers.Adam(), metrics = ['accuracy'])
# mlp.summary()

In [12]:
# mlp.fit(training, validation_data = validation, epochs = 64, batch_size = 64, callbacks = Callbacks)

In [13]:
# propertyType_validation_pred_MLP = np.argmax(mlp.predict(validation), axis = 1)
# confusion_matrix(propertyType_validation_fac, propertyType_validation_pred_MLP)

## Convolutional Neural Network (CNN) Model

In [14]:
cnn = Sequential([
                  Rescaling(1. / 255, input_shape = (height, width, 3)),
                  Conv2D(8, 4, padding = 'same', activation = tf.nn.leaky_relu),
                  MaxPooling2D(pool_size = (2, 2)),
                  Conv2D(4, 4, padding = 'same', activation = tf.nn.leaky_relu),
                  MaxPooling2D(pool_size = (2, 2)),
                  Flatten(),
                  Dense(16, activation = tf.nn.leaky_relu),
                  Dropout(.25),
                  Dense(
                        len(labels.cat.categories), 
                        activation = tf.nn.softmax, 
                        kernel_initializer = initializers.RandomNormal(),
                        bias_initializer = initializers.Zeros(),
                        kernel_regularizer = regularizers.L1(1e-4),
                        bias_regularizer = regularizers.L1(1),
                        activity_regularizer = regularizers.L1(1)
                       )
                ])
cnn.compile(loss = loss, optimizer = optimizers.Adam(), metrics = ['accuracy'])
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 64, 64, 3)         0         
                                                                 
 conv2d (Conv2D)             (None, 64, 64, 8)         392       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 8)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 4)         516       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 16, 16, 4)        0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1024)              0

In [15]:
cnn.fit(training, validation_data = validation, epochs = 64, batch_size = 64, callbacks = Callbacks)

Epoch 1/64
302/302 [==============================] - 5s 7ms/step - loss: 2.3408 - accuracy: 0.3204 - val_loss: 2.2870 - val_accuracy: 0.3814 - lr: 0.0010
Epoch 2/64
302/302 [==============================] - 2s 6ms/step - loss: 2.2835 - accuracy: 0.3838 - val_loss: 2.2640 - val_accuracy: 0.3952 - lr: 0.0010
Epoch 3/64
302/302 [==============================] - 2s 7ms/step - loss: 2.2656 - accuracy: 0.4001 - val_loss: 2.2519 - val_accuracy: 0.4155 - lr: 0.0010
Epoch 4/64
302/302 [==============================] - 2s 6ms/step - loss: 2.2473 - accuracy: 0.4167 - val_loss: 2.2419 - val_accuracy: 0.4230 - lr: 0.0010
Epoch 5/64
302/302 [==============================] - 2s 6ms/step - loss: 2.2376 - accuracy: 0.4227 - val_loss: 2.2360 - val_accuracy: 0.4269 - lr: 0.0010
Epoch 6/64
302/302 [==============================] - 2s 5ms/step - loss: 2.2251 - accuracy: 0.4330 - val_loss: 2.2366 - val_accuracy: 0.4303 - lr: 0.0010
Epoch 7/64
302/302 [==============================] - 2s 6ms/step - lo

In [16]:
propertyType_validation_pred_CNN = np.argmax(cnn.predict(validation), axis = 1)
confusion_matrix(propertyType_validation_fac, propertyType_validation_pred_CNN)

130/130 [==============================] - 1s 2ms/step


array([[384,  49, 416, 138],
       [ 84, 224, 143, 243],
       [283,  58, 660, 213],
       [189, 109, 424, 515]])